# Приложение для сжатия и суммаризации текста
<a href='https://github.com/E7HYN3/sirius_ai2024' target="_blank"><img alt='GitHub' src='https://img.shields.io/badge/github-100000?style=flat&logo=GitHub&logoColor=white&labelColor=black&color=black'/></a> <a target="_blank" href="https://colab.research.google.com/github/https://colab.research.google.com/drive/1j-FdgRfbcW3zYJEJRwUT_oVaL-eiLuNm?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a><br>
___
Реализация дополнительного задания по сжатию текста при помощи ИИ


In [ ]:
# @title Установка зависимостей
!pip install ollama streamlit st-copy-to-clipboard
!curl https://ollama.ai/install.sh | sh
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

In [ ]:
# @title Загрузка LLM моделей
import os
os.environ.update({'OLLAMA_HOST': '0.0.0.0'})
!nohup ollama serve &
!ollama pull gemma2:9b
!ollama pull llama3.2:3b

In [ ]:
# @title Исходный код приложения
%%writefile app.py
import streamlit as st
from st_copy_to_clipboard import st_copy_to_clipboard
import ollama


# Функция для выполнения запроса к модели Ollama
def summarize_with_ollama(text, model_name, compression_level):
    """
    Выполняет запрос к Ollama для сжатия текста.

    Args:
        text (str): Текст для сжатия.
        model_name (str): Название модели Ollama.
        compression_level (str): Уровень сжатия ("Сильное" или "Слабое").

    Returns:
        str: Сжатый текст.
    """
    prompt_template = "Сократи текст строго до {}:\n{}"
    if compression_level == "Сильное":
        prompt = prompt_template.format("одного или двух коротких предложений", text)
    else:
        prompt = prompt_template.format("одного краткого абзаца", text)
    response = ollama.generate(model=model_name, prompt=prompt)
    return response["response"]


# Интерфейс Streamlit
st.title("Приложение для сжатия и суммаризации текста")

# Поле ввода текста
input_text = st.text_area("Введите текст")

# Список моделей для выбора
available_models = [model["name"] for model in ollama.list()["models"]]
selected_model = st.selectbox("Выберите модель для сжатия", available_models)

# Выбор уровня сжатия
compression_type = st.radio("Выберите уровень сжатия", ("Сильное", "Слабое"))

# Кнопка запуска
if st.button("Сжать текст"):
    result = summarize_with_ollama(input_text, selected_model, compression_type)
    # Вывод результата
    st.title("Результат")
    st.write(result)
    st_copy_to_clipboard(
        result, before_copy_label="Копировать", after_copy_label="Скопировано!"
    )


In [ ]:
# @title Запуск приложения
!nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &
!ollama serve &>/content/ollama.txt &
!grep -Eo 'https://.*\.trycloudflare.com' nohup.out | tail -1| xargs -I {} echo "Your tunnel url {}"
!streamlit run app.py &>/content/streamlit.txt &